In [1]:
import os
import re
import numpy as np
import pandas as pd

In [2]:
iid = True
num_users = 100
frac = 0.1
local_ep = 1

dataset = 'mnist'
# dataset = 'cifar10'
# dataset = 'cifar100'

shard_per_user = 10

if dataset == 'mnist':
    model = 'mlp'
    rd_lg = 100
    rd_fed = 800 + int(rd_lg*0.15)
elif dataset == 'cifar10':
    model = 'cnn'
    rd_lg = 100
    rd_fed = 1800 + int(rd_lg*0.04)
elif dataset == 'cifar100':
    model = 'cnn'
    rd_fed = 1800
    rd_lg = 100

In [3]:
base_dir = './save/{}/{}_iid{}_num{}_C{}_le{}/shard{}/'.format(
    dataset, model, iid, num_users, frac, local_ep, shard_per_user)
runs = os.listdir(base_dir)

In [4]:
acc_fed = np.zeros(len(runs))
acc_local_localtest = np.zeros(len(runs))
acc_local_newtest_avg = np.zeros(len(runs))
acc_local_newtest_ens = np.zeros(len(runs))
lg_metrics = {}

for idx, run in enumerate(runs):
    # FedAvg
    base_dir_fed = os.path.join(base_dir, "{}/fed".format(run))
    results_path_fed = os.path.join(base_dir_fed, "results.csv")
    df_fed = pd.read_csv(results_path_fed)

    acc_fed[idx] = df_fed.loc[rd_fed - 1]['best_acc']
    
    # LocalOnly
    base_dir_local = os.path.join(base_dir, "{}/local".format(run))
    results_path_local = os.path.join(base_dir_local, "results.csv")
    df_local = pd.read_csv(results_path_local)

    acc_local_localtest[idx] = df_local.loc[0]['acc_test_local']
    acc_local_newtest_avg[idx] = df_local.loc[0]['acc_test_avg']
    if 'acc_test_ens' in df_local.columns:
        acc_local_newtest_ens[idx] = df_local.loc[0]['acc_test_ens']
    else:
        acc_local_newtest_ens[idx] = df_local.loc[0]['acc_test_ens_avg']
    
    # LGFed
    base_dir_lg = os.path.join(base_dir, "{}/lg/".format(run))
    lg_runs = os.listdir(base_dir_lg)
    for lg_run in lg_runs:
        results_path_lg = os.path.join(base_dir_lg, "{}/results.csv".format(lg_run))
        df_lg = pd.read_csv(results_path_lg)
        
        load_fed = int(re.split('best_|.pt', lg_run)[1])
        if load_fed not in lg_metrics.keys():
            lg_metrics[load_fed] = {'acc_local': np.zeros(len(runs)),
                                    'acc_avg': np.zeros(len(runs)),
                                    'acc_ens': np.zeros(len(runs))}
            
        x = df_lg.loc[rd_lg]['best_acc_local']
        lg_metrics[load_fed]['acc_local'][idx] = x
        idx_acc_local = df_lg[df_lg['best_acc_local'] == x].index[0]
        lg_metrics[load_fed]['acc_avg'][idx] = df_lg.loc[idx_acc_local]['acc_test_avg']
        if 'acc_test_ens' in df_lg.columns:
            lg_metrics[load_fed]['acc_ens'][idx] = df_lg['acc_test_ens'].values[-1]
        else:
            lg_metrics[load_fed]['acc_ens'][idx] = df_lg['acc_test_ens_avg'].values[-1]

In [5]:
columns = ["Run", "Local Test", "New Test (avg)", "New Test (ens)", "FedAvg Rounds", "LG Rounds"]
results = []

In [6]:
str_acc_local_localtest = "{:.2f} +- {:.2f}".format(acc_local_localtest.mean(), acc_local_localtest.std())
str_acc_local_newtest_avg = "{:.2f} +- {:.2f}".format(acc_local_newtest_avg.mean(), acc_local_newtest_avg.std())
str_acc_local_newtest_ens = "{:.2f} +- {:.2f}".format(acc_local_newtest_ens.mean(), acc_local_newtest_ens.std())

print("localonly:\t", str_acc_local_localtest)
print("localonly_avg:\t", str_acc_local_newtest_avg)
print("localonly_ens:\t", str_acc_local_newtest_ens)

results.append(["LocalOnly", str_acc_local_localtest, str_acc_local_newtest_avg, str_acc_local_newtest_ens, 0, 0])

localonly:	 88.03 +- 0.37
localonly_avg:	 86.24 +- 0.87
localonly_ens:	 91.15 +- 0.27


In [7]:
for lg_run in sorted(lg_metrics.keys()):
    x = ["LG-FedAvg"]
    print(lg_run)
    for array in ['acc_local', 'acc_avg', 'acc_ens']:
        mean = lg_metrics[lg_run][array].mean()
        std = lg_metrics[lg_run][array].std()
        str_acc = "{:.2f} +- {:.2f}".format(mean, std)
        print("{}:\t{}".format(array, str_acc))
        
        x.append(str_acc)
    x.append(lg_run)
    x.append(rd_lg)
    results.append(x)

300
acc_local:	97.45 +- 0.12
acc_avg:	97.50 +- 0.10
acc_ens:	97.49 +- 0.09
400
acc_local:	97.59 +- 0.08
acc_avg:	97.61 +- 0.08
acc_ens:	97.62 +- 0.08
500
acc_local:	97.78 +- 0.13
acc_avg:	97.82 +- 0.14
acc_ens:	97.82 +- 0.13
600
acc_local:	97.84 +- 0.10
acc_avg:	97.86 +- 0.08
acc_ens:	97.87 +- 0.10
700
acc_local:	97.85 +- 0.09
acc_avg:	97.88 +- 0.09
acc_ens:	97.89 +- 0.09
800
acc_local:	97.91 +- 0.10
acc_avg:	97.93 +- 0.07
acc_ens:	97.93 +- 0.07


In [8]:
str_acc_fed = "{:.2f} +- {:.2f}".format(acc_fed.mean(), acc_fed.std())
print("fed:\t", str_acc_fed)
results.append(["FedAvg", str_acc_fed, str_acc_fed, str_acc_fed, rd_fed, 0])

fed:	 97.93 +- 0.08


In [10]:
pd.DataFrame(results, columns=columns).set_index("Run")

,Local Test,New Test (avg),New Test (ens),FedAvg Rounds,LG Rounds
Run,,,,,
LocalOnly,88.03 +- 0.37,86.24 +- 0.87,91.15 +- 0.27,0,0
LG-FedAvg,97.45 +- 0.12,97.50 +- 0.10,97.49 +- 0.09,300,100
LG-FedAvg,97.59 +- 0.08,97.61 +- 0.08,97.62 +- 0.08,400,100
LG-FedAvg,97.78 +- 0.13,97.82 +- 0.14,97.82 +- 0.13,500,100
LG-FedAvg,97.84 +- 0.10,97.86 +- 0.08,97.87 +- 0.10,600,100
LG-FedAvg,97.85 +- 0.09,97.88 +- 0.09,97.89 +- 0.09,700,100
LG-FedAvg,97.91 +- 0.10,97.93 +- 0.07,97.93 +- 0.07,800,100
FedAvg,97.93 +- 0.08,97.93 +- 0.08,97.93 +- 0.08,815,0
